In [2]:
import os
import shutil
import glob

In [3]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

Mounted at /content/drive


In [4]:
!unzip '/content/drive/MyDrive/barath/Dataset.zip'

Archive:  /content/drive/MyDrive/barath/Dataset.zip
  inflating: Dataset/test/Russula/289_q_9DtnUpSv4.jpg  
  inflating: Dataset/test/Russula/344_OIUTQ-Vo4is.jpg  
  inflating: Dataset/test/Russula/043_a_5lm55zCVc.jpg  
  inflating: Dataset/test/Russula/231_BQOv5Indrvw.jpg  
  inflating: Dataset/test/Russula/208_j0DtfZk8VuM.jpg  
  inflating: Dataset/test/Russula/139_myC25hDxlYM.jpg  
  inflating: Dataset/test/Russula/127_VbgJEufLv7I.jpg  
  inflating: Dataset/test/Russula/141_VQiBUAnjD3U.jpg  
  inflating: Dataset/test/Russula/021_L5l_KuergPQ.jpg  
  inflating: Dataset/test/Russula/169_yrB2szRIBCk.jpg  
  inflating: Dataset/test/Russula/057_jOjOWsmPwSs.jpg  
  inflating: Dataset/test/Russula/094_KEJSmxOfGTE.jpg  
  inflating: Dataset/test/Russula/093_mpXQiQhYX9A.jpg  
  inflating: Dataset/test/Russula/032_CA8Ox7CbMzI.jpg  
  inflating: Dataset/test/Russula/024_6ZiLaxdJnHs.jpg  
  inflating: Dataset/test/Russula/042_Y4YrMX7E-qc.jpg  
  inflating: Dataset/test/Russula/193_0iPR4s0EgAE.jp

In [5]:
imageSize = [224, 224]

In [6]:
train_data_dir = '/content/Dataset/train'
valid_data_dir = '/content/Dataset/test'

In [7]:
import os
import PIL
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img 
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [8]:
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                          
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
training_set = train_datagen.flow_from_directory(train_data_dir,
                                                target_size = (224, 224),
                                                batch_size = 32,
                                                class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(valid_data_dir,
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 911 images belonging to 3 classes.
Found 306 images belonging to 3 classes.


##Model Training

In [10]:

xception = Xception (input_shape=imageSize + [3], weights='imagenet',include_top=False)

83683744/83683744 [==============================] - 1s 0us/step


In [11]:
for layer in xception.layers: 
  layer.trainable = False

In [12]:
x = Flatten() (xception.output)
x = Flatten() (xception.output)

In [13]:
prediction = Dense(3, activation='softmax')(x)

In [14]:
model = Model(inputs=xception.input, outputs=prediction)

In [15]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [18]:
r = model.fit( 
    training_set, 
    validation_data=test_set, 
    epochs=15, 
    steps_per_epoch=len(training_set)//5, 
    validation_steps=len (test_set)//5
)

Epoch 1/15
5/5 [==============================] - 5s 986ms/step - loss: 0.4673 - accuracy: 0.9187 - val_loss: 0.9250 - val_accuracy: 0.8906
Epoch 2/15
5/5 [==============================] - 6s 1s/step - loss: 0.6461 - accuracy: 0.9062 - val_loss: 0.5432 - val_accuracy: 0.9062
Epoch 3/15
5/5 [==============================] - 4s 880ms/step - loss: 0.4625 - accuracy: 0.9187 - val_loss: 0.2654 - val_accuracy: 0.9375
Epoch 4/15
5/5 [==============================] - 6s 1s/step - loss: 0.4290 - accuracy: 0.9125 - val_loss: 0.8921 - val_accuracy: 0.8594
Epoch 5/15
5/5 [==============================] - 4s 915ms/step - loss: 0.7715 - accuracy: 0.8625 - val_loss: 0.5246 - val_accuracy: 0.8281
Epoch 6/15
5/5 [==============================] - 4s 887ms/step - loss: 0.9625 - accuracy: 0.8813 - val_loss: 0.3785 - val_accuracy: 0.9688
Epoch 7/15
5/5 [==============================] - 5s 1s/step - loss: 1.0159 - accuracy: 0.8531 - val_loss: 1.0626 - val_accuracy: 0.8125
Epoch 8/15
5/5 [=============

In [19]:
model.save('mushroom.h5')

##Model Testing

In [20]:
training_set.class_indices

{'Boletus': 0, 'Lactarius': 1, 'Russula': 2}

In [21]:
img=image.load_img(r"/content/Dataset/test/Boletus/0430_EfZRbD-1yAI.jpg", target_size=(224 , 224))

In [22]:
x=image.img_to_array(img)

In [23]:
x.shape

(224, 224, 3)

In [24]:
arr=np.array(['Boletus','Lactarius','Russula'])

In [25]:
import numpy as np
x=np.expand_dims (x,axis=0)
img_data=preprocess_input(x) 
img_data.shape

(1, 224, 224, 3)

In [26]:
output=np.argmax(model.predict(img_data), axis=1)
print(arr[output])

1/1 [==============================] - 1s 1s/step
['Boletus']


In [27]:
img1=image.load_img(r"/content/Dataset/test/Russula/032_CA8Ox7CbMzI.jpg", target_size=(224 , 224))
y=image.img_to_array(img1)
y=np.expand_dims (y,axis=0)
img1_data=preprocess_input(y) 
img1_data.shape
output=np.argmax(model.predict(img1_data), axis=1)
print(arr[output])

1/1 [==============================] - 0s 34ms/step
['Russula']


In [28]:
img2=image.load_img(r"/content/Dataset/test/Lactarius/0317_fcNIf7KZfRA.jpg", target_size=(224 , 224))
z=image.img_to_array(img2)
z=np.expand_dims (z,axis=0)
img2_data=preprocess_input(z) 
img2_data.shape
output=np.argmax(model.predict(img2_data), axis=1)
print(arr[output])

1/1 [==============================] - 0s 25ms/step
['Lactarius']


In [29]:
img3=image.load_img(r"/content/Dataset/test/Lactarius/0317_fcNIf7KZfRA.jpg", target_size=(224 , 224))
g=image.img_to_array(img3)
g=np.expand_dims (g,axis=0)
img3_data=preprocess_input(g) 
img3_data.shape
output=np.argmax(model.predict(img3_data), axis=1)
print(arr[output])

1/1 [==============================] - 0s 24ms/step
['Lactarius']
